## Implement Residual Network

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
EPOCHS = 10

## Residual Unit

In [3]:
class ResidualUnit(tf.keras.Model):
    def __init__(self, filter_in, filter_out, kernel_size):
        super(ResidualUnit, self).__init__()
        # Pre-activation
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        # Identity mapping
        if filter_in == filter_out:
            self.identity = lambda x:x
        else:
            self.identity = tf.keras.layers.Conv2D(filter_out, (1,1), padding='same')
            
    def call(self, x, training=False, mask=None):
        h = self.bn1(x, training=training)
        h = tf.nn.relu(h)
        h = self.conv1(h)
        
        h = self.bn2(h, training=training)
        h = tf.nn.relu(h)
        h = self.conv2(h)
        
        return self.identity(x) + h

## Residual Layer

In [4]:
class ResnetLayer(tf.keras.Model):
    def __init__(self, filter_in, filters, kernel_size):
        super(ResnetLayer, self).__init__()
        # 16, [32, 32, 32, 32]
        self.sequence = []
        for f_in, f_out in zip([filter_in] + list(filters), filters):
            # [16] + [32, 32, 32]
            # -> [16, 32, 32, 32], [32, 32, 32]
            self.sequence.append(ResidualUnit(f_in, f_out, kernel_size))
            
    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            x = unit(x, training=training)
        return x

## ResNet model

In [12]:
class ResNet(tf.keras.Model):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(8, (3, 3), padding='same', activation='relu') # 28x28x8
        
        self.res1 = ResnetLayer(8, (16, 16), (3, 3)) # 28x28x16
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2)) # 14x14x16
        
        self.res2 = ResnetLayer(16, (32, 32), (3, 3)) # 14x14x32
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2)) # 7x7x32
        
        self.res3 = ResnetLayer(32, (64, 64), (3, 3)) # 7x7x64
        self.flatten = tf.keras.layers.Flatten() # 7x7x64=
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x, training=False, mask=None):
        x = self.conv1(x)
        x = self.res1(x, training)
        x = self.pool1(x)
        
        x = self.res2(x, training)
        x = self.pool2(x)
        
        x = self.res3(x, training)
        
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

## Training/test loop

In [17]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)
    
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## Prepare dataset (MNIST)

In [14]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Model Define

In [15]:
# create model
model = ResNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_loss')

In [18]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
        
    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1, 
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))

Epoch 1, Loss: 0.1412116438150406, Accuracy: 96.01667022705078, Test Loss: 0.10512052476406097, Test Accuracy: 96.9000015258789
Epoch 2, Loss: 0.1046801507472992, Accuracy: 97.06749725341797, Test Loss: 0.08145838975906372, Test Accuracy: 97.79000091552734
Epoch 3, Loss: 0.0849836990237236, Accuracy: 97.60833740234375, Test Loss: 0.06796219199895859, Test Accuracy: 98.10333251953125
Epoch 4, Loss: 0.07337149232625961, Accuracy: 97.93083190917969, Test Loss: 0.06044449284672737, Test Accuracy: 98.31999969482422
Epoch 5, Loss: 0.0649155005812645, Accuracy: 98.1719970703125, Test Loss: 0.05397352948784828, Test Accuracy: 98.49200439453125
Epoch 6, Loss: 0.05937538668513298, Accuracy: 98.32805633544922, Test Loss: 0.053012385964393616, Test Accuracy: 98.52166748046875
Epoch 7, Loss: 0.0547369085252285, Accuracy: 98.45333099365234, Test Loss: 0.050967615097761154, Test Accuracy: 98.60143280029297
Epoch 8, Loss: 0.05075378715991974, Accuracy: 98.56812286376953, Test Loss: 0.0479285754263401,